In [1]:
import pandas as pd
import numpy as np
import matplotlib as mplt
import keras 

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

In [2]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")

In [3]:
x = train_data.iloc[:,1:]
y = train_data.iloc[:,0]

In [4]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test =  train_test_split(x,y,test_size=0.1)

In [5]:
x_train = x_train.values.reshape(-1,28,28,1)
x_test = x_test.values.reshape(-1,28,28,1)

In [6]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [7]:
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255
datagen.fit(x_train)

In [8]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [11]:
model = Sequential()
model.add(Conv2D(filters = 32,activation='relu',strides=1,padding = 'same', kernel_size=(4,4),data_format='channels_last',input_shape =(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32,activation='relu',strides=1,padding='same',kernel_size=(4,4),data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=2,pool_size=(3,3),padding='valid'))
model.add(Dropout(0.25))          

model.add(Conv2D(filters = 64,activation = 'relu',strides=1,padding='same',kernel_size=(4,4),data_format='channels_last'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64,activation = 'relu',strides=1,padding='same',kernel_size=(4,4),data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=2,pool_size=(3,3),padding='valid'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

In [12]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999 )

In [13]:
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])

In [15]:
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [16]:
batch_size = 32
epochs = 40
history = model.fit(datagen.flow(x_train,y_train,batch_size=batch_size),epochs=epochs,
validation_data=(x_test,y_test),verbose=1,steps_per_epoch=x_train.shape[0]//batch_size,callbacks=[reduce_lr])

Epoch 1/40


1181/1181 [==============================] - 42s 34ms/step - loss: 0.3355 - accuracy: 0.9034 - val_loss: 0.0905 - val_accuracy: 0.9750 - lr: 0.0010
Epoch 2/40
1181/1181 [==============================] - 40s 34ms/step - loss: 0.1262 - accuracy: 0.9631 - val_loss: 0.0846 - val_accuracy: 0.9757 - lr: 9.0000e-04
Epoch 3/40
1181/1181 [==============================] - 42s 35ms/step - loss: 0.1004 - accuracy: 0.9707 - val_loss: 0.0362 - val_accuracy: 0.9888 - lr: 8.1000e-04
Epoch 4/40
1181/1181 [==============================] - 42s 36ms/step - loss: 0.0842 - accuracy: 0.9762 - val_loss: 0.0536 - val_accuracy: 0.9864 - lr: 7.2900e-04
Epoch 5/40
1181/1181 [==============================] - 43s 37ms/step - loss: 0.0737 - accuracy: 0.9779 - val_loss: 0.0321 - val_accuracy: 0.9890 - lr: 6.5610e-04
Epoch 6/40
1181/1181 [==============================] - 43s 36ms/step - loss: 0.0596 - accuracy: 0.9828 - val_loss: 0.0446 - val_accuracy: 0.9869 - lr: 5.9049e-04
Epoch 7/40
1181/1181 [==